import libraries

In [2]:
import pandas as pd
import numpy as np





from scipy.stats import spearmanr

import importlib

import clean_up_merge_order
importlib.reload(clean_up_merge_order)

<module 'clean_up_merge_order' from 'c:\\Users\\Michaelr\\Desktop\\co-occurence\\clean_up_merge_order.py'>

In [3]:

df = clean_up_merge_order.clean_and_merge_data('data/raw/project_otu.csv', 'data/raw/taxon_tab.csv')
df.head()

,Full_Taxonomy,BjSa-1,BjSa-2,BjSa-4,BjSa-5,BjSa-6,BjSa-7,Replant-1,Replant-2,Replant-3,Replant-5,Replant-6,Replant-7,Kingdom,Phylum,Class,Order,Family,Genus,Species
0,k__Bacteria;p__;c__;o__;f__;g__;s__,30191,36210,36393,27191,26308,27531,24781,28313,26240,27181,24850,24850,k__Bacteria,p__,c__,o__,f__,g__,s__
1,k__Bacteria;p__4484-113;c__4484-113;o__4484-11...,23,48,63,34,40,40,4,30,5,0,12,0,k__Bacteria,p__4484-113,c__4484-113,o__4484-113,f__JAFGCY01,g__JADZFO01,s__JADZFO01 sp020849855
2,k__Bacteria;p__ARS69;c__ARS69;o__ARS69;f__ARS6...,2,0,35,2,0,0,54,35,151,65,38,33,k__Bacteria,p__ARS69,c__ARS69,o__ARS69,f__ARS69,g__JAGQIB01,s__JAGQIB01 sp041393395
3,k__Bacteria;p__ARS69;c__ARS69;o__ARS69;f__ARS6...,0,0,0,0,0,2,0,2,0,4,0,0,k__Bacteria,p__ARS69,c__ARS69,o__ARS69,f__ARS69,g__JALGZR01,s__JALGZR01 sp025774805
4,k__Bacteria;p__Acidobacteriota;c__;o__;f__;g__...,136,134,247,242,307,119,322,306,470,321,373,413,k__Bacteria,p__Acidobacteriota,c__,o__,f__,g__,s__


filter initially

In [4]:
metadata = pd.read_csv('data/raw/16s_sample_metadata_file.tsv', sep='\t', index_col=0)

In [5]:

target_treatment = 'BjSa'  # <-- change to the exact label you want
treat_samples = metadata[metadata['treatment'] == 'BjSa'].index



In [6]:
df_genus = clean_up_merge_order.make_taxon_df(df, taxon_level='Genus')

In [7]:
taxonomy_levels = ["Kingdom", "Phylum", "Class", "Order", "Family","Genus"]
df_genus["Full_Taxonomy"] = df_genus[taxonomy_levels].agg(";".join, axis=1)


In [8]:
df_genus.head()

,Genus,BjSa-1,BjSa-2,BjSa-4,BjSa-5,BjSa-6,BjSa-7,Replant-1,Replant-2,Replant-3,Replant-5,Replant-6,Replant-7,Kingdom,Phylum,Class,Order,Family,Full_Taxonomy
0,g__,206906,220155,193849,237182,238689,258052,176093,194156,199079,191432,167398,178846,k__Bacteria,p__,c__,o__,f__,k__Bacteria;p__;c__;o__;f__;g__
1,g__JADZFO01,23,48,63,34,40,40,4,30,5,0,12,0,k__Bacteria,p__4484-113,c__4484-113,o__4484-113,f__JAFGCY01,k__Bacteria;p__4484-113;c__4484-113;o__4484-11...
2,g__JAGQIB01,2,0,35,2,0,0,54,35,151,65,38,33,k__Bacteria,p__ARS69,c__ARS69,o__ARS69,f__ARS69,k__Bacteria;p__ARS69;c__ARS69;o__ARS69;f__ARS6...
3,g__JALGZR01,0,0,0,0,0,2,0,2,0,4,0,0,k__Bacteria,p__ARS69,c__ARS69,o__ARS69,f__ARS69,k__Bacteria;p__ARS69;c__ARS69;o__ARS69;f__ARS6...
4,g__JAIBAN01,284,258,231,254,270,242,243,181,201,144,169,179,k__Bacteria,p__Acidobacteriota,c__Blastocatellia,o__Chloracidobacteriales,f__Chloracidobacteriaceae,k__Bacteria;p__Acidobacteriota;c__Blastocatell...


In [9]:

sample_cols = [c for c in df_genus.columns if c in treat_samples]
df_treat = df_genus.loc[:, ["Full_Taxonomy"] + sample_cols]


only transpose once!

In [10]:
df_treat = df_treat.T

# Make taxonomy the column names and drop the taxonomy row
df_treat.columns = df_treat.loc["Full_Taxonomy"].astype(str).values
df_treat = df_treat.drop(index="Full_Taxonomy").apply(pd.to_numeric, errors="coerce")



In [11]:
df_treat.head(10)

,k__Bacteria;p__;c__;o__;f__;g__,k__Bacteria;p__4484-113;c__4484-113;o__4484-113;f__JAFGCY01;g__JADZFO01,k__Bacteria;p__ARS69;c__ARS69;o__ARS69;f__ARS69;g__JAGQIB01,k__Bacteria;p__ARS69;c__ARS69;o__ARS69;f__ARS69;g__JALGZR01,k__Bacteria;p__Acidobacteriota;c__Blastocatellia;o__Chloracidobacteriales;f__Chloracidobacteriaceae;g__JAIBAN01,k__Bacteria;p__Acidobacteriota;c__Blastocatellia;o__Chloracidobacteriales;f__JADJPF01;g__JADJPF01,k__Bacteria;p__Acidobacteriota;c__Blastocatellia;o__Chloracidobacteriales;f__JADJPF01;g__JBFLVQ01,k__Bacteria;p__Acidobacteriota;c__Blastocatellia;o__DASRDY01;f__DASRDY01;g__DASRDY01,k__Bacteria;p__Acidobacteriota;c__Blastocatellia;o__HR10;f__J050;g__J050,k__Bacteria;p__Acidobacteriota;c__Blastocatellia;o__HR10;f__J050;g__JANWZE01,...,k__Bacteria;p__Vulcanimicrobiota;c__Xenobia;o__RGZZ01;f__RGZZ01;g__RGZZ01,k__Bacteria;p__Vulcanimicrobiota;c__Xenobia;o__Xenobiales;f__JADMJV01;g__JADMJV01,k__Bacteria;p__Vulcanimicrobiota;c__Xenobia;o__Xenobiales;f__JADMJV01;g__JAEXNA01,k__Bacteria;p__Vulcanimicrobiota;c__Xenobia;o__Xenobiales;f__Xenobiaceae;g__Xenobium,k__Bacteria;p__Zhuqueibacterota;c__Zhuqueibacteria;o__Residuimicrobiales;f__Residuimicrobiaceae;g__Residuimicrobium,k__Bacteria;p__Zixibacteria;c__MSB-5A5;o__DSPP01;f__DSPP01;g__DSPP01,k__Bacteria;p__Zixibacteria;c__MSB-5A5;o__GN15;f__FEB-12;g__DAOSXA01,k__Bacteria;p__Zixibacteria;c__MSB-5A5;o__GN15;f__FEB-12;g__JACZWU01,k__Bacteria;p__Zixibacteria;c__MSB-5A5;o__GN15;f__FEB-12;g__JAJRTA01,k__Bacteria;p__Zixibacteria;c__MSB-5A5;o__WJJR01;f__WJJR01;g__JACQFV01
BjSa-1,206906,23,2,0,284,0,70,9,0,0,...,17,0,0,0,0,3,0,0,0,0
BjSa-2,220155,48,0,0,258,0,83,12,0,0,...,12,0,0,0,2,0,0,0,0,0
BjSa-4,193849,63,35,0,231,3,71,14,0,6,...,11,2,24,0,0,21,5,0,2,0
BjSa-5,237182,34,2,0,254,0,69,21,0,0,...,20,4,0,10,0,0,0,0,0,0
BjSa-6,238689,40,0,0,270,0,97,19,2,0,...,20,4,0,14,0,0,0,0,0,3
BjSa-7,258052,40,0,2,242,0,64,21,0,0,...,19,16,0,15,0,22,0,0,0,0


In [118]:
df_treat.to_csv('data/processed/df_treat_bjsa.csv', index=False)

In [19]:
min_occurence = 0.10 *len(df_treat)  # Set minimum occurence threshold to 10% of samples
df_filtered = df_treat.loc[:, (df_treat > 0).sum(axis=0) >= min_occurence]


In [20]:
corr_matrix, p_matrix = spearmanr(df_filtered, axis=0)

In [21]:
print(corr_matrix)

[[ 1.         -0.23190841 -0.77151675 ... -0.65465367 -0.65465367
   0.3927922 ]
 [-0.23190841  1.          0.         ...  0.66421116  0.66421116
   0.        ]
 [-0.77151675  0.          1.         ...  0.70710678  0.70710678
  -0.42426407]
 ...
 [-0.65465367  0.66421116  0.70710678 ...  1.          1.
  -0.2       ]
 [-0.65465367  0.66421116  0.70710678 ...  1.          1.
  -0.2       ]
 [ 0.3927922   0.         -0.42426407 ... -0.2        -0.2
   1.        ]]


In [22]:
corr_df = pd.DataFrame(corr_matrix, index=df_filtered.columns, columns=df_filtered.columns)
p_df = pd.DataFrame(p_matrix, index=df_filtered.columns, columns=df_filtered.columns)

In [23]:
np.fill_diagonal(corr_df.values, 0)

In [29]:
corr_df.head(10)

,k__Bacteria;p__;c__;o__;f__;g__,k__Bacteria;p__4484-113;c__4484-113;o__4484-113;f__JAFGCY01;g__JADZFO01,k__Bacteria;p__ARS69;c__ARS69;o__ARS69;f__ARS69;g__JAGQIB01,k__Bacteria;p__ARS69;c__ARS69;o__ARS69;f__ARS69;g__JALGZR01,k__Bacteria;p__Acidobacteriota;c__Blastocatellia;o__Chloracidobacteriales;f__Chloracidobacteriaceae;g__JAIBAN01,k__Bacteria;p__Acidobacteriota;c__Blastocatellia;o__Chloracidobacteriales;f__JADJPF01;g__JADJPF01,k__Bacteria;p__Acidobacteriota;c__Blastocatellia;o__Chloracidobacteriales;f__JADJPF01;g__JBFLVQ01,k__Bacteria;p__Acidobacteriota;c__Blastocatellia;o__DASRDY01;f__DASRDY01;g__DASRDY01,k__Bacteria;p__Acidobacteriota;c__Blastocatellia;o__HR10;f__J050;g__J050,k__Bacteria;p__Acidobacteriota;c__Blastocatellia;o__HR10;f__J050;g__JANWZE01,...,k__Bacteria;p__Vulcanimicrobiota;c__Vulcanimicrobiia;o__Vulcanimicrobiales;f__Vulcanimicrobiaceae;g__Vulcanimicrobium,k__Bacteria;p__Vulcanimicrobiota;c__Xenobia;o__RGZZ01;f__RGZZ01;g__RGZZ01,k__Bacteria;p__Vulcanimicrobiota;c__Xenobia;o__Xenobiales;f__JADMJV01;g__JADMJV01,k__Bacteria;p__Vulcanimicrobiota;c__Xenobia;o__Xenobiales;f__JADMJV01;g__JAEXNA01,k__Bacteria;p__Vulcanimicrobiota;c__Xenobia;o__Xenobiales;f__Xenobiaceae;g__Xenobium,k__Bacteria;p__Zhuqueibacterota;c__Zhuqueibacteria;o__Residuimicrobiales;f__Residuimicrobiaceae;g__Residuimicrobium,k__Bacteria;p__Zixibacteria;c__MSB-5A5;o__DSPP01;f__DSPP01;g__DSPP01,k__Bacteria;p__Zixibacteria;c__MSB-5A5;o__GN15;f__FEB-12;g__DAOSXA01,k__Bacteria;p__Zixibacteria;c__MSB-5A5;o__GN15;f__FEB-12;g__JAJRTA01,k__Bacteria;p__Zixibacteria;c__MSB-5A5;o__WJJR01;f__WJJR01;g__JACQFV01
k__Bacteria;p__;c__;o__;f__;g__,0.000000,-0.231908,-0.771517,0.654654,0.028571,-0.654654,-0.257143,0.724714,0.392792,-0.654654,...,-0.135225,0.753702,0.794461,-0.654654,0.941124,-0.130931,-0.030359,-0.654654,-0.654654,0.392792
k__Bacteria;p__4484-113;c__4484-113;o__4484-113;f__JAFGCY01;g__JADZFO01,-0.231908,0.000000,0.000000,0.000000,-0.637748,0.664211,0.405840,-0.014706,0.000000,0.664211,...,-0.788893,-0.602941,-0.044781,0.664211,-0.184812,0.398527,0.154010,0.664211,0.664211,0.000000
k__Bacteria;p__ARS69;c__ARS69;o__ARS69;f__ARS69;g__JAGQIB01,-0.771517,0.000000,0.000000,-0.424264,-0.308607,0.707107,-0.185164,-0.203523,-0.424264,0.707107,...,0.438178,-0.391390,-0.286039,0.707107,-0.557452,-0.424264,0.229539,0.707107,0.707107,-0.424264
k__Bacteria;p__ARS69;c__ARS69;o__ARS69;f__ARS69;g__JALGZR01,0.654654,0.000000,-0.424264,0.000000,-0.392792,-0.200000,-0.654654,0.531369,-0.200000,-0.200000,...,-0.309839,0.132842,0.674200,-0.200000,0.695608,-0.200000,0.695608,-0.200000,-0.200000,-0.200000
k__Bacteria;p__Acidobacteriota;c__Blastocatellia;o__Chloracidobacteriales;f__Chloracidobacteriaceae;g__JAIBAN01,0.028571,-0.637748,-0.308607,-0.392792,0.000000,-0.654654,0.371429,-0.521794,0.392792,-0.654654,...,0.304256,0.318874,-0.441367,-0.654654,-0.151794,0.130931,-0.516100,-0.654654,-0.654654,0.392792
k__Bacteria;p__Acidobacteriota;c__Blastocatellia;o__Chloracidobacteriales;f__JADJPF01;g__JADJPF01,-0.654654,0.664211,0.707107,-0.200000,-0.654654,0.000000,0.130931,-0.132842,-0.200000,1.000000,...,-0.309839,-0.664211,-0.134840,1.000000,-0.417365,-0.200000,0.417365,1.000000,1.000000,-0.200000
k__Bacteria;p__Acidobacteriota;c__Blastocatellia;o__Chloracidobacteriales;f__JADJPF01;g__JBFLVQ01,-0.257143,0.405840,-0.185164,-0.654654,0.371429,0.130931,0.000000,-0.463817,0.654654,0.130931,...,-0.439480,-0.144943,-0.441367,0.130931,-0.333947,0.392792,-0.576818,0.130931,0.130931,0.654654
k__Bacteria;p__Acidobacteriota;c__Blastocatellia;o__DASRDY01;f__DASRDY01;g__DASRDY01,0.724714,-0.014706,-0.203523,0.531369,-0.521794,-0.132842,-0.463817,0.000000,0.132842,-0.132842,...,0.051450,0.632353,0.940403,-0.132842,0.831655,-0.398527,0.092406,-0.132842,-0.132842,0.132842
k__Bacteria;p__Acidobacteriota;c__Blastocatellia;o__HR10;f__J050;g__J050,0.392792,0.000000,-0.424264,-0.200000,0.392792,-0.200000,0.654654,0.132842,0.000000,-0.200000,...,-0.309839,0.531369,0.269680,-0.200000,0.417365,-0.

In [30]:
corr_df.shape

(2084, 2084)

In [31]:
corr_df.to_csv('data/processed/bjsa_genus_spearman_correlation_matrix.csv')
p_df.to_csv('data/processed/bjsa_genus_spearman_pvalue_matrix.csv')

In [25]:

# Find index (row, col) of maximum value
max_idx = corr_df.values.argmax()
row, col = np.unravel_index(max_idx, corr_df.shape)

# Get ASV names and the value
asv1 = corr_df.index[row]
asv2 = corr_df.columns[col]
max_value = corr_df.iloc[row, col]

print("Highest co-occurrence:")
print(f"{asv1}  <-->  {asv2}   =  {max_value}")


Highest co-occurrence:
k__Bacteria;p__;c__;o__;f__;g__  <-->  k__Bacteria;p__Acidobacteriota;c__Vicinamibacteria;o__Vicinamibacterales;f__UBA2999;g__WHSN01   =  1.0


In [26]:

# Convert to NumPy array
M = corr_df.values.copy()

# Zero the diagonal (optional, but usually wanted)
np.fill_diagonal(M, 0)

# Use only the upper triangle (i < j) to avoid duplicating pairs
tri_u = np.triu_indices_from(M, k=1)

# Extract the upper triangle values
vals = M[tri_u]

# Find the indices of the top 10 largest values
top10_idx = np.argpartition(vals, -10)[-10:]   # FAST
top10_sorted = top10_idx[np.argsort(vals[top10_idx])[::-1]]  # sort descending

# Build output: list of (ASV_i, ASV_j, value)
top_10 = []
for idx in top10_sorted:
    i = tri_u[0][idx]
    j = tri_u[1][idx]
    top_10.append((corr_df.index[i], corr_df.columns[j], vals[idx]))

for triple in top_10:
    print(triple)


('k__Bacteria;p__Pseudomonadota;c__Alphaproteobacteria;o__UBA9219;f__UBA9219;g__CAIXLI01', 'k__Bacteria;p__Pseudomonadota;c__Gammaproteobacteria;o__Nevskiales;f__Nevskiaceae;g__Solimonas', np.float64(1.0))
('k__Bacteria;p__Pseudomonadota;c__Alphaproteobacteria;o__UBA9219;f__UBA9219;g__CAIXLI01', 'k__Bacteria;p__Pseudomonadota;c__Gammaproteobacteria;o__Legionellales;f__Legionellaceae;g__JACETI01', np.float64(1.0))
('k__Bacteria;p__Pseudomonadota;c__Alphaproteobacteria;o__UBA9219;f__UBA9219;g__CAIXLI01', 'k__Bacteria;p__Pseudomonadota;c__Gammaproteobacteria;o__Burkholderiales;f__SG8-41;g__DSOT01', np.float64(1.0))
('k__Bacteria;p__Acidobacteriota;c__Terriglobia;o__Acidiferrales;f__UBA7541;g__Palsa-147', 'k__Bacteria;p__Pseudomonadota;c__Gammaproteobacteria;o__Burkholderiales;f__UBA11063;g__Aquella', np.float64(1.0))
('k__Bacteria;p__Pseudomonadota;c__Alphaproteobacteria;o__UBA9219;f__UBA9219;g__CAIXLI01', 'k__Bacteria;p__Pseudomonadota;c__Gammaproteobacteria;o__Burkholderiales;f__JAEUOS0

values equal 1

In [27]:
# Convert to NumPy array
M = corr_df.values.copy()

# Zero the diagonal (optional, but usually wanted)
np.fill_diagonal(M, 0)

# Use only the upper triangle (i < j) to avoid duplicating pairs
tri= np.triu_indices_from(M, k=1)


# All entries in the upper triangle where corr = 1.0
mask = M[tri] == 1.0

# Extract ASV index pairs + values
asv_i = corr_df.index[tri[0][mask]]
asv_j = corr_df.columns[tri[1][mask]]
vals  = M[tri][mask]

# Build a clean DataFrame
pairs_eq1 = pd.DataFrame({
    "ASV_i": asv_i,
    "ASV_j": asv_j,
    "corr": vals
})

print("Number of ASV pairs with corr = 1.0:", len(pairs_eq1))
pairs_eq1.sample(20)


Number of ASV pairs with corr = 1.0: 18530


,ASV_i,ASV_j,corr
9718,k__Bacteria;p__Bacteroidota;c__Bacteroidia;o__...,k__Bacteria;p__Planctomycetota;c__Planctomycet...,1.0
8807,k__Bacteria;p__Bacteroidota;c__Bacteroidia;o__...,k__Bacteria;p__Omnitrophota;c__Koll11;o__Zapsy...,1.0
1591,k__Bacteria;p__Acidobacteriota;c__Terriglobia;...,k__Bacteria;p__Chloroflexota;c__Anaerolineae;o...,1.0
2831,k__Bacteria;p__Actinomycetota;c__Acidimicrobii...,k__Bacteria;p__Myxococcota;c__Polyangia;o__Hal...,1.0
17288,k__Bacteria;p__Planctomycetota;c__Planctomycet...,k__Bacteria;p__Pseudomonadota;c__Alphaproteoba...,1.0
3333,k__Bacteria;p__Actinomycetota;c__Acidimicrobii...,k__Bacteria;p__Armatimonadota;c__Armatimonadia...,1.0
10979,k__Bacteria;p__Chloroflexota;c__Anaerolineae;o...,k__Bacteria;p__Hydrogenedentota;c__Hydrogenede...,1.0
10274,k__Bacteria;p__Bdellovibrionota;c__UBA1018;o__...,k__Bacteria;p__Pseudomonadota;c__Alphaproteoba...,1.0
8347,k__Bacteria;p__Bacillota;c__Negativicutes;o__S...,k__Bacteria;p__Pseudomonadota;c__Alphaproteoba...,1.0
9229,k__Bacteria;p__Bacteroidota;c__Bacteroidia;o__...,k__Bacteria;p__Bdellovibrionota;c__Bacteriovor...,1.0


In [41]:
df.loc["2ca089c1df78f3c2b2f194ed34433aa2"]

BjSa-1       31
BjSa-2       41
BjSa-4        0
BjSa-5       36
BjSa-6       39
BjSa-7       44
Past-1        0
Past-2        0
Past-3        0
Past-4        0
Past-5       41
Past-6       52
Replant-1     0
Replant-2     0
Replant-3     0
Replant-5     0
Replant-6     0
Replant-7     0
Name: 2ca089c1df78f3c2b2f194ed34433aa2, dtype: int64

In [42]:
df.loc["80406f4229979c59b190101a1fcc678f"]

BjSa-1       192
BjSa-2       246
BjSa-4       130
BjSa-5       231
BjSa-6       245
BjSa-7       258
Past-1        37
Past-2         0
Past-3         0
Past-4         0
Past-5        42
Past-6        31
Replant-1    128
Replant-2    167
Replant-3    179
Replant-5    122
Replant-6     90
Replant-7    121
Name: 80406f4229979c59b190101a1fcc678f, dtype: int64